In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
import sys
import string
import pickle
from tqdm import tqdm_notebook
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold

Для обучения классификатора был использован <a href="http://study.mokoron.com/">размеченный корпус</a> коротких постов (из твиттера), поэтому обобщающая способность модели при применении к текстам из других источников может страдать.

_Рубцова Ю. Автоматическое построение и анализ корпуса коротких текстов (постов микроблогов) для задачи разработки и тренировки тонового классификатора //Инженерия знаний и технологии семантического веба. – 2012. – Т. 1. – С. 109-116._

In [2]:
DATA_PATH = './data/'

In [3]:
df_positive = pd.read_csv(os.path.join(DATA_PATH, 'positive.csv'), sep=';', header=None)
print(df_positive.shape)
df_positive.head()

(114911, 12)


,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [4]:
df_negative = pd.read_csv(os.path.join(DATA_PATH, 'negative.csv'), sep=';', header=None)
print(df_negative.shape)
df_negative.head()

(111923, 12)


,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [5]:
df_positive = df_positive[[3]]
df_positive['y'] = 1
df_negative = df_negative[[3]]
df_negative['y'] = 0

df_train = pd.concat([df_positive, df_negative]).reset_index()
df_train['y'].value_counts()

1    114911
0    111923
Name: y, dtype: int64

In [6]:
def preprocess_text_ignore_non_letters(text):
    russian_letters = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    text = text.lower().strip()
    text = ''.join(char for char in text if char in russian_letters or char.isspace())
    text = [word for word in text.split() if word[0] != '@' and word]
    text = ' '.join(text)
    return text

def save_model(model, filepath):
    with open(filepath, 'wb') as f:
        f.write(pickle.dumps(model))

In [7]:
df_train['words_raw'] = df_train[3].apply(preprocess_text_ignore_non_letters)
df_train.drop(columns=['index', 3], inplace=True)
df_train.head()

,y,words_raw
0,1,хоть я и школота но поверь у нас то же самое о...
1,1,да всетаки он немного похож на него но мой мал...
2,1,ну ты идиотка я испугалась за тебя
3,1,кто то в углу сидит и погибает от голода а мы ...
4,1,вот что значит страшилка но блинпосмотрев все ...


In [8]:
y_train = df_train['y']

bow = CountVectorizer(ngram_range=(1, 2), min_df=2, max_df=0.5)
# tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.5)
# bow_hash = HashingVectorizer(ngram_range=(1, 2), n_features=1048576 // 8)
X_train = bow.fit_transform(df_train['words_raw'])

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
model = LogisticRegression(solver='saga')

scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv)
print(scores)
print(scores.mean())

In [ ]:
model.fit(X_train, y_train)

In [ ]:
def predict_proba(text, vectorizer):
    text = preprocess_text_ignore_non_letters(text)
    text = vectorizer.transform([text])
    return model.predict_proba(text)[0, 1]

In [ ]:
# интерактивное "тестирование"
for sample_text in [' ', 'не круто', 'круто',
                    'у меня умерла собака',
                    'ваш телефон сломался',
                    'наконец-то!!!',
                    'ужасный продукт',
                    'крутой продукт',
                    'не так уж и плохо']:
    print(f'{sample_text:32s} - этот текст позитивный на {int(100 * predict_proba(sample_text, bow))}%')

In [ ]:
vectorizer_filepath = './model/vec_bow.pickle'
model_filepath = './model/logit.pickle'

save_model(bow, vectorizer_filepath)
save_model(model, model_filepath)

In [ ]:
# Шаблон для запуска Flask-сервера с формой
from flask import Flask, request, render_template


vectorizer_filepath = './model/vec_bow.pickle'
model_filepath = './model/logit.pickle'

app = Flask(__name__)
app.vectorizer = pickle.load(open(vectorizer_filepath, 'rb'))
app.model = pickle.load(open(model_filepath, 'rb'))
app.score = 0
app.text = ''

def predict_score(text):
    text_bow = app.vectorizer.transform([text])
    return int(100 * app.model.predict_proba(text_bow)[0, 1])

@app.route('/')
def input_text_form():
    return render_template('input_form.html', score_value=0, default_text=app.text)

@app.route('/', methods=['POST'])
def my_form_post():
    text = request.form['text']
    app.text = text
    processed_text = preprocess_text_ignore_non_letters(text)
    app.score = predict_score(processed_text)
    return render_template('input_form.html', score_value=app.score, default_text=app.text)

app.run()